In [1]:
%cd images

/Users/lukazphelps/Desktop/SchoolCode/DS2.2/Final_Project/images


In [2]:
# from PyInstaller.utils.hooks import collect_submodules
# hiddenimports = collect_submodules('tensorflow_core')
import glob
import pandas as pd
import numpy as np
import os
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from PIL import Image
from keras.preprocessing.image import img_to_array
from tensorflow.keras.models import Sequential, load_model
from keras.utils import np_utils
from tensorflow.keras import layers
from keras.layers.core import Dense, Activation
from keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dropout
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import img_to_array
# print(Sequential())

Using TensorFlow backend.


In [3]:
files = glob.glob("Fire_images/*.*")
ls_fire = []
for filepath in files:
    ls_fire.append(['Fire_images', os.path.basename(filepath), '1'])

In [4]:
files = glob.glob("Normal_Images_1/*.*")
ls_normal = []
for filepath in files:
    ls_normal.append(['Normal_Images_1', os.path.basename(filepath), '0'])

In [5]:
df_fire = pd.DataFrame(ls_fire, columns=['Folder', 'filename', 'label'])

In [6]:
df_normal = pd.DataFrame(ls_normal, columns=['Folder', 'filename', 'label'])

In [7]:
df_fire.head()

,Folder,filename,label
0,Fire_images,dsc_01001.jpg,1
1,Fire_images,burning-charcoal-briquettes.jpg,1
2,Fire_images,Chimney-Fire3.jpg,1
3,Fire_images,canada-wildfire.jpg,1
4,Fire_images,Living_Room_Camp_Fire.jpg,1


In [8]:
df_normal.head()

,Folder,filename,label
0,Normal_Images_1,1526t.jpg,0
1,Normal_Images_1,23e90181fcef550cffae7c4ff77e566d.jpg,0
2,Normal_Images_1,157_TypicalOffice-1024x881.jpg,0
3,Normal_Images_1,aes7.jpg,0
4,Normal_Images_1,7bfa705a0e9058e8e33494bbe76a330c.jpg,0


In [9]:
df = pd.concat([df_fire, df_normal]).reset_index().drop('index', axis=1) # combining the dataframes
df

,Folder,filename,label
0,Fire_images,dsc_01001.jpg,1
1,Fire_images,burning-charcoal-briquettes.jpg,1
2,Fire_images,Chimney-Fire3.jpg,1
3,Fire_images,canada-wildfire.jpg,1
4,Fire_images,Living_Room_Camp_Fire.jpg,1
...,...,...,...
258,Normal_Images_1,amazing-modern-living-room-design-with-white-b...,0
259,Normal_Images_1,813e6b10-848b-478e-a471-480f14eeb271.1.10.jpg,0
260,Normal_Images_1,201504-connect-norway-horizontal-1b.jpg,0
261,Normal_Images_1,690x380-Lake-Tahoe-Summer-Emerald-Bay.jpg,0


In [10]:
df_train, df_test = train_test_split(df, test_size=0.2, random_state=0)


In [11]:
img = Image.open(os.path.join(df['Folder'][260], df['filename'][260]))
print(img.size)
image_red = img.resize((1024, 1024))
image = img_to_array(image_red)

(930, 581)


In [12]:
print("size: " + str(image_red.size))


size: (1024, 1024)


In [13]:
image.shape

(1024, 1024, 3)

In [14]:
def data_gen(df, batch_size):
    
    while True:
        x_batch = np.zeros((batch_size, 1024, 1024, 3))
        y_batch = np.zeros((batch_size, 1))


        for chunk_index in range(len(df)//batch_size):
            start_chunk_index = chunk_index*batch_size
            end_chunk_index = (chunk_index+1)*batch_size


            i = 0
            for folder, filename, label in zip(df['Folder'].values[start_chunk_index:end_chunk_index], df['filename'].values[start_chunk_index:end_chunk_index], df['label'].values[start_chunk_index:end_chunk_index]):
                img = Image.open(os.path.join(folder, filename))
                image_red = img.resize((1024, 1024))
                try:
                    x_batch[i] = img_to_array(image_red)
                except:
                    print(f'this file is currupted: {os.path.join(folder, filename)}')
                    os.remove(os.path.join(folder, filename))

                y_batch[i] = label
                i += 1


            yield x_batch, np_utils.to_categorical(y_batch, 2)     

In [15]:
df['filename']


0                                          dsc_01001.jpg
1                        burning-charcoal-briquettes.jpg
2                                      Chimney-Fire3.jpg
3                                    canada-wildfire.jpg
4                              Living_Room_Camp_Fire.jpg
                             ...                        
258    amazing-modern-living-room-design-with-white-b...
259        813e6b10-848b-478e-a471-480f14eeb271.1.10.jpg
260              201504-connect-norway-horizontal-1b.jpg
261            690x380-Lake-Tahoe-Summer-Emerald-Bay.jpg
262    AAEAAQAAAAAAAAJqAAAAJDAwNDUwNDAwLWUwYTAtNDlmNy...
Name: filename, Length: 263, dtype: object

In [16]:
model = tf.keras.Sequential()
model.add(layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(1024, 1024, 3)))
model.add(layers.Conv2D(32, kernel_size=(3,3), activation='relu'))  
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Dropout(0.5))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(2, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
             optimizer=tf.keras.optimizers.Adam(0.01),
             metrics = ['accuracy'])

In [18]:
# PATH = '/content/gdrive/MyDrive/DS2.2/Final_Project/images'
# batch_size = 50 #batch size = 8*x  [50 is a good start]
# epochs = 2

# def data_gen(df, batch_size):
#     while True:
#         x_batch = np.zeros((batch_size, 1024, 1024, 3))
#         y_batch = np.zeros((batch_size, 1))
#         for j in range(int(len(df)/batch_size)):
#             for m, k in zip(df['filename'].values[j*batch_size:(j+1)*batch_size], df['label'].values[j*batch_size:(j+1)*batch_size]):
#                 b = 0
#                 img = Image.open(f'{df["Folder"][b]}/{df["filename"][b]}')
#                 image_red = img.resize((1024, 1024))
#                 x_batch[b] = img_to_array(image_red)
#                 y_batch[b] = df['label'][b]
#                 b += 1
#             yield (x_batch, y_batch)

# #model.fit_generator is depricated, updated
# model.fit(data_gen(df_train, batch_size=batch_size), steps_per_epoch=len(df_train) // batch_size, epochs=epochs)

batch_size = 50
model.fit(data_gen(df_train, batch_size=batch_size), steps_per_epoch=len(df_train) // batch_size, epochs=2, verbose=1)

Epoch 1/2
4/4 [==============================] - 502s 125s/step - loss: 53513.2383 - accuracy: 0.5650
Epoch 2/2
4/4 [==============================] - 421s 105s/step - loss: 7649.0508 - accuracy: 0.5400


In [19]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 1022, 1022, 32)    896       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 1020, 1020, 32)    9248      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 510, 510, 32)      0         
_________________________________________________________________
dropout (Dropout)            (None, 510, 510, 32)      0         
_________________________________________________________________
flatten (Flatten)            (None, 8323200)           0         
_________________________________________________________________
dense (Dense)                (None, 64)                532684864 
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0

In [21]:
model_2 = tf.keras.Sequential()
model_2.add(layers.Conv2D(16, kernel_size=(3, 3), activation='relu', input_shape=(1024, 1024, 3)))
model_2.add(layers.MaxPooling2D(pool_size=(2, 2)))
model_2.add(layers.Dropout(0.5))
model_2.add(layers.Flatten())
model_2.add(layers.Dense(4, activation='relu'))
model_2.add(layers.Dropout(0.5))
model_2.add(layers.Dense(2, activation='sigmoid'))

model_2.compile(loss='binary_crossentropy',
             optimizer=tf.keras.optimizers.Adam(0.01),
             metrics = ['accuracy'])

In [22]:
model_2.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 1022, 1022, 16)    448       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 511, 511, 16)      0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 511, 511, 16)      0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 4177936)           0         
_________________________________________________________________
dense_2 (Dense)              (None, 4)                 16711748  
_________________________________________________________________
dropout_3 (Dropout)          (None, 4)                 0         
_________________________________________________________________
dense_3 (Dense)              (None, 2)                

In [23]:
batch_size = 32
model_2.fit(data_gen(df_train, batch_size=batch_size), steps_per_epoch=len(df_train) // batch_size, epochs=2, verbose=1)

Epoch 1/2
6/6 [==============================] - 27s 5s/step - loss: 4.5007 - accuracy: 0.5677
Epoch 2/2
6/6 [==============================] - 28s 5s/step - loss: 0.6893 - accuracy: 0.5781


In [ ]:
#I tried my best, minimized loss, but can't figure out accuracy